<a href="https://colab.research.google.com/github/oliverzannino/oliverza/blob/main/2025Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="font-family:Impact,Arial;font-size:30px;">37005 Fundamentals of Derivative Security Pricing - Spring 2025</h1>
<h1 style="font-family:Impact,Arial;font-size:45px;">Assignment Part 2</h1>
<h2 style="font-family:Arial;">Erik Schl&ouml;gl</h2>
<p><small> School of Mathematical &amp; Physical Sciences<br>
University of Technology Sydney
</small></p>
<p>
<a href="mailto:Erik.Schlogl@uts.edu.au?Subject=37000 JIT" target="_blank">
<small><font color=MediumVioletRed>Erik.Schlogl@uts.edu.au</font></small></a>
</p>
<hr style="height:5px;border:none;color:#333;background-color:#333;" />

# Standard normal CDF

## Task 1
Write a program in Python that asks a user for a number `x`, and then outputs the value of the cumulative distribution function of the standard normal distribution at `x`. *(1 mark)*
## Example

`Enter x: 0.3
The value of the standard normal CDF at 0.3 is 0.6179114221889526`

**Hint:** You'll want to use the `input(...)` and `print(...)` functions for this. Recall that `input(...)` returns a string, which must be converted to a floating point number using `float(...)`. If we do `from scipy.stats import norm`, then the cumulative distribution function of the standard normal distribution can be accessed as `norm.cdf(...)`.

In [1]:
from scipy.stats import norm

def CDF_UI():
    x = float(input("Enter x: "))
    print(f'The value of the standard normal CDF at {x} is {norm.cdf(x)}')

In [2]:
CDF_UI()

Enter x: .3
The value of the standard normal CDF at 0.3 is 0.6179114221889526


# Black/Scholes formula in Python
The Black/Scholes price of a European call option expiring at time $T$ with strike price $K$ is
$$
C(S,t)=SN(d_1)−Ke^{−r(T−t)}N(d_2)
$$
where $S$ is the current price of the underlying asset, $t$ the current time and $r$ the continuously compounded riskfree interest rate. $N(d)$ denotes the cumulative distribution function of the standard normal distribution, and
$$
\begin{eqnarray}
d_1 &=& \frac{\ln\frac{S}K+(r+\frac12\sigma^2)(T−t)}{\sigma\sqrt{T-t}}\\
d_2 &=& d_1−\sigma\sqrt{T-t}
\end{eqnarray}
$$
where the $\sigma$ denotes the volatility of the underlying asset.

Similarly, the price of a European put option expiring at time $T$ with strike price $K$ is
$$
P(S,t)=Ke^{−r(T−t)}N(−d_2)−SN(−d_1)
$$
## Task 2
Using the scaffold provided, write a Python function which calculates the Black/Scholes price of the option, where the function takes six arguments (in this order): $S$, $K$, $\sigma$, $r$, $T$ and a 1 for a call or -1 for a put. <em>(2 marks)</em>

## Example:

`Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.3
Enter continuously compounded interest rate: 0.05
Enter the time to maturity: 2
Enter 1 for call or -1 for put option: 1
The option price is: 21.193735255280203`

## Another example (a put option):

`Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.3
Enter continuously compounded interest rate: 0.05
Enter the time to maturity: 2
Enter 1 for call or -1 for put option: -1
The option price is: 11.677477058876157`

In [3]:
import numpy as np
from scipy.stats import norm

def BlackScholes(S,K,sgm,r,T,callput):
    d1 = (np.log(S/K)+(r+(sgm**2)/2)*T)/(sgm*np.sqrt(T))
    d2 = d1-sgm*np.sqrt(T)
    if callput==1:
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
sigma = float(input('Enter the volatility: '))
interest = float(input('Enter continuously compounded interest rate: '))
maturity = float(input('Enter the time to maturity: '))
callput = int(input('Enter 1 for call or -1 for put option: '))
print('The option price is: ')
print(BlackScholes(stock,strike,sigma,interest,maturity,callput))


Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.3
Enter continuously compounded interest rate: 0.05
Enter the time to maturity: 2
Enter 1 for call or -1 for put option: 1
The option price is: 
21.193735255280203


# Monte Carlo simulation
The goal is to calculate the expected value of a function $f(\cdot)$ of a random variable $x$, where the distribution of $x$ is given by the probability density $\psi(x)$, i.e.1
$$
E[f(x)]=\int_{-\infty}^{\infty}f(x)\psi(x)dx
$$

## Outline of the Monte Carlo simulation
1. Establish a procedure for drawing variates $x$ from the target distribution $\psi(x)$.
2. Initialise the variables:
   RunningSum = 0
   RunningSumSquared = 0
   $i=1$
3. Draw a realisation $x_i$ from the target distribution.
4. Add $f(x_i)$ to RunningSum and  $(f(x_i))^2$ to RunningSumSquared.
5. Increment the counter $i$. If $i$ is less than the maximum number of iterations, go to step 3.
6. Calculate the simulated mean by dividing RunningSum by the total number of iterations.
7. Calculate the variance of the simulations by dividing RunningSumSquared by the total number of iterations and subtracting the square of the mean.

## Error estimation for Monte Carlo methods
By the Central Limit Theorem, we know that for a large number $N$ of simulations, the simulation mean $X_N$ is approximately normally distributed, with standard deviation
$$
\sqrt{\frac{\sigma^2}N}
$$
where the simulation variance is an estimate for $\sigma^2$.

Thus, if there is no bias, the simulation mean is normally distributed around the target value with a standard deviation, which decreases with $\sqrt{N}$.

A 95% confidence interval for the target value is therefore approximately given by
$$
\left[X_N-2\sqrt{\frac{\sigma^2}N};X_N+2\sqrt{\frac{\sigma^2}N}\right]
$$

<font color='red'>**Monte Carlo simulation without error bounds is meaningless!**</font>

The NumPy function `random.standard_normal()` returns a random variate drawn from the standard normal distribution, while `random.standard_normal(n)` returns `n` such variates in a Numpy array:

In [4]:
import numpy as np
print(np.random.standard_normal())
n = 5
print(np.random.standard_normal(n))

-0.8468212793182863
[ 1.01119352 -1.34679425  0.04025612 -0.91037729 -1.22219835]




```
# This is formatted as code
```

Recall that a standard normal random variable can be converted into a normal random variable of desired mean and standard deviation by multiplying by the standard deviation and adding the mean.

## Monte Carlo pricing of a Black/Scholes call option
In the Black/Scholes model, the price of the underlying stock follows Geometric Brownian motion, with the dynamics under the risk-neutral measure given by
$$S(T)=S(t)\exp\left\{\left(r−\frac12\sigma^2\right)(T−t)+\sigma(W(T)−W(t))\right\}$$
Recall that the time 0 price of a European call option (and analogously the put option) expiring at time $T$ with strike price $K$ can be expressed as the expectation under the risk-neutral measure of
$$C=E\left[e^{−rT}\max(0,S(T)−K)\right]$$
Thus we can write a Python function which calculates the Monte Carlo estimate `MC` for the Black/Scholes price of the option and the standard deviation `MCstd` of the simulation mean, where the function takes seven arguments (in this order): $S$, $K$, $\sigma$, $r$, $T$, a 1 for a call or -1 for a put, and $n$, the number of sampling iterations of the Monte Carlo algorithm:

In [5]:
def BlackScholesMC(S,K,sgm,r,T,callput,n):
    w = np.random.standard_normal(n)
    ST=S*np.exp((r-0.5*sgm**2)*T+sgm*np.sqrt(T)*w)
    payoff=callput*(ST-K)
    payoff=payoff*(payoff>0)
    MC=np.exp(-r*T)*np.mean(payoff)
    MCstd=np.exp(-r*T)*np.std(payoff)/np.sqrt(n)
    return MC, MCstd

To run this code with user inputs:

In [6]:
stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
sigma = float(input('Enter the volatility: '))
interest = float(input('Enter continuously compounded interest rate: '))
maturity = float(input('Enter the time to maturity: '))
callput = int(input('Enter 1 for call or -1 for put option: '))
n = int(input('Enter the number of simulations: '))
MC, MCstd = BlackScholesMC(stock,strike,sigma,interest,maturity,callput,n)
print('The MC estimate for the option price is: ')
print(MC)
print('The 2 standard deviation confidence interval for the option price is: ')
print(MC-2*MCstd,MC+2*MCstd)

Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.3
Enter continuously compounded interest rate: 0.05
Enter the time to maturity: 2
Enter 1 for call or -1 for put option: 1
Enter the number of simulations: 10000
The MC estimate for the option price is: 
21.818171533197617
The 2 standard deviation confidence interval for the option price is: 
21.099403803433365 22.53693926296187


## Task 3
Consider a European call option with maturity $T$ on a foreign asset $\tilde S$, which pays (in domestic currency) at time $T$
$$
\max[0,X(T)\tilde S(T)-X(0)\tilde S(T)]
$$
Assume that $\tilde S$ follows the dynamics (under the foreign risk--neutral measure) of
\begin{eqnarray*}
d\tilde S(t) &=& \tilde S(t)(\tilde rdt+\xi d\tilde W(t))
\end{eqnarray*}
and the exchange rate $X$ (in units of domestic currency per unit of foreign currency) follows the dynamics (under the domestic risk--neutral measure) of
\begin{eqnarray*}
dX(t) &=& X(t)((r-\tilde r)dt+\sigma dW(t))
\end{eqnarray*}
with $W(t)$ and $\tilde W(t)$ two--dimensional standard Brownian motions under the respective measures, $r$ and $\tilde r$ scalar constants, and $\xi$ and $\sigma$ are two--dimensional constant vectors.
### Task 3(a)
Using the scaffold provided, write a Python function which calculates the price of the option, where the function takes eight arguments (in this order): $\tilde S(0)$, $X(0)$, $r$, $\tilde r$, $T$, the Black/Scholes volatility of $\tilde S$, the Black/Scholes volatility of $X$ and the correlation coefficient between the dynamics of $\tilde S$ and $X$. <em>(9 marks)</em>


Derivation:

\begin{align}
V(t, X(t), \tilde{S}(t)) &= \mathbb{E}_\beta\left[ e^{-r(T-t)} \max\left(0, X(T) \tilde{S}(T) - X(0) \tilde{S}(T) \right) \mid \mathcal{F}_t \right] \\
&= e^{-r(T-t)} \mathbb{E}_\beta \left[ X(T) \tilde{S}(T) \, \mathbb{I}_{\{ X(T) \tilde{S}(T) > X(0) \tilde{S}(T) \}} \mid \mathcal{F}_t \right] \notag \\
&\quad - e^{-r(T-t)} \mathbb{E}_\beta \left[ X(0) \tilde{S}(T) \, \mathbb{I}_{\{ X(T) \tilde{S}(T) > X(0) \tilde{S}(T) \}} \mid \mathcal{F}_t \right] \notag
\end{align}

1) Using Ito's Lemma, we get:
\begin{align*}
\tilde{S}(T) &= \tilde{S}(t) \exp\left( \left( \tilde{r} - \frac{1}{2} \|\xi\|^2 \right)(T - t) + \xi \left( \tilde{W}(T) - \tilde{W}(t) \right) \right) \\
\ln \tilde{S}(T) \mid \mathcal{F}_t &\overset{Q_{\tilde{\beta}}}{\sim} \mathcal{N} \left( \ln \tilde{S}(t) + \left( r - \frac{1}{2} \|\xi\|^2 \right)(T - t),\|\xi\|^2 (T - t) \right)
\end{align*}

\begin{align*}
X(T) &= X(t) \exp\left( \left( r - \tilde{r} - \frac{1}{2} \|\sigma\|^2 \right)(T - t) + \sigma \left(W(T) - W(t) \right) \right) \\
\ln X(T) \mid \mathcal{F}_t &\overset{Q_\beta}{\sim} \mathcal{N} \left( \ln X(t) + \left( r - \tilde{r} - \frac{1}{2} \|\sigma\|^2 \right)(T - t),\ \|\sigma\|^2 (T - t) \right)
\end{align*}

2) First term:

\begin{align*}
& e^{-r(T-t)} \, \mathbb{E}_{\beta} \left[ X(T) \tilde{S}(T) \mathbb{I}_{\{ X(T) \tilde{S}(T) > X(0) \tilde{S}(T) \}} \mid \mathcal{F}_t \right] \\
& \text{Recall that } \frac{X(T) \tilde{S}(T)}{\beta(T)} \text{ is a martingale under } \mathbb{Q}_\beta \\
& \text{so the Radon-Nikodym derivative for the change of measure to } \mathbb{Q}_{\tilde{S}} \text{ is given by:} \\
& \quad \frac{d \mathbb{Q}_{\tilde{S}}}{d \mathbb{Q}_\beta} = \frac{X(T) \tilde{S}(T) \beta(t)}{\beta(T) X(t) \tilde{S}(t)} \\
& \text{which corresponds to the exponential martingale:} \\
& \left. \frac{d \mathbb{Q}_{\tilde{S}}}{d \mathbb{Q}_\beta} \right|_{\mathcal{F}_t}
= \exp\left( \int_t^T (\sigma + \xi) \, dW_\beta(u) - \frac{1}{2} \int_t^T (\sigma + \xi)^2 \, du \right) \\
& \text{and under this new measure } \mathbb{Q}_{\tilde{S}}, \text{ the process has volatility } (\sigma + \xi) \\
\\
& \text{By Girsanov's theorem, the Brownian motion under } \mathbb{Q}_{\tilde{S}} \text{ is:} \\
& \quad dW_{\tilde{S}}(t) = dW_\beta(t) - (\sigma + \xi)dt \\
& \text{so under } \mathbb{Q}_{\tilde{S}}: \\
& \quad X(T) = X(t) \exp \left( \left( r - \tilde{r} + \frac{1}{2} \|\sigma\|^2 + \sigma^{\top} \xi \right)(T - t)
+ \sigma \left( W_{\tilde{S}}(T) - W_{\tilde{S}}(t) \right) \right) \\
& \text{which implies:} \\
& \quad \ln X(T) \mid \mathcal{F}_t \overset{\mathbb{Q}_{\tilde{S}}}{\sim} \mathcal{N} \left( \ln X(t) + \left( r - \tilde{r} + \frac{1}{2} \|\sigma\|^2 + \sigma^{\top} \xi \right)(T - t), \|\sigma\|^2 (T - t) \right)
\end{align*}

\begin{align*}
& \text{Inserting the Radon-Nikodym derivative into the expectation and changing measure to } \mathbb{Q}_{\tilde{S}} \\
& e^{-r(T - t)} \, \mathbb{E}_{\tilde{S}} \left[ \left. X(T) \tilde{S}(T) \mathbb{I}_{\{ X(T) \tilde{S}(T) > X(0) \tilde{S}(T) \}} \frac{\beta(T) X(t) \tilde{S}(t)}{X(T) \tilde{S}(T) \beta(t)} \right| \mathcal{F}_t \right] \\
&= X(t) \tilde{S}(t) \mathbb{Q}_{\tilde{S}} \left( X(T) > X(0) \right) \\
&= X(t) \tilde{S}(t) N \left( \frac{ \ln \left( \frac{X(t)}{X(0)} \right) + \left( r - \tilde{r} + \frac{1}{2} \|\sigma\|^2 + \sigma^{\top} \xi \right)(T - t) }{ \sigma \sqrt{T - t} } \right)
\end{align*}

3) Second term:

\begin{align*}
& e^{-r(T - t)} \, \mathbb{E}_\beta \left[ X(0) \tilde{S}(T) \, \mathbb{I}_{\{ X(T)\tilde{S}(T) > X(0)\tilde{S}(T) \}} \mid \mathcal{F}_t \right] \\
& \text{When attempting to change measure to remove the } \tilde{S}(T) \text{ term, note that:} \\
& \frac{\tilde{S}(T)}{\beta(T)} \quad \text{does not have a martingale form, as we are measuring foreign/domestic.}
\end{align*}

Recall the form of the exponential martingale:
\begin{align*}
\exp\left( \int_t^T \xi \, dW_\beta(u) - \frac{1}{2} \int_t^T \|\xi\|^2 \, du \right)
\end{align*}

So the eventual Radon-Nikodym derivative must be of the form:
\begin{align*}
\frac{\tilde{S}(T)}{\beta(T)} \frac{\beta(t)}{\tilde{S}(t)}  C
= \exp\left( \int_t^T \xi \, dW_\beta(u) - \frac{1}{2} \int_t^T \|\xi\|^2 \, du \right)
\end{align*}

which has the exponential martingale form.

Firstly, we want to change measure from $\tilde{\beta} \rightarrow \beta$ for the process $\tilde{S}(T)$

\begin{aligned}
& \text{So the Radon-Nikodym derivative for the change of measure from } \mathbb{Q}_{\tilde{\beta}} \text{ to } \mathbb{Q}_\beta \text{ is given by} \\
& \frac{d \mathbb{Q}_\beta}{d \mathbb{Q}_{\tilde{\beta}}} \bigg|_{\mathcal{F}_t}
= \frac{\tilde{B}(T) X(T)}{\beta(T)} \frac{\beta(t)}{\beta(t) X(t)}
\end{aligned}

Using Girsanov's theorem:
\begin{aligned}
d W_{\tilde{\beta}}(t) &= d W_\beta(t) - \sigma\, dt
\end{aligned}

\begin{align*}
\text{Substituting into the expression for } \tilde{S}(T) \text{ under } \mathbb{Q}_\beta\text{:} \\
\tilde{S}(T) &= \tilde{S}(t) \exp\left( \left( \tilde{r} - \frac{1}{2} \|\xi\|^2 - \sigma^\top \xi \right)(T - t)
+ \xi \left( W_\beta(T) - W_\beta(t) \right) \right)
\end{align*}


Then, solving for C:

\begin{aligned}
& e^{\left( \tilde{r} - \frac{1}{2} \|\xi\|^2 - \sigma^{\top} \xi - r \right)(T - t)
+ \xi \left(W_\beta(T) - W_\beta(t)\right)} e^{-r(T - t)} C \\
=\, & e^{ -\frac{1}{2} \|\xi\|^2 (T - t) + \xi \left(W_\beta(T) - W_\beta(t)\right) }
\end{aligned}

\begin{aligned}
& C = \frac{1}{e^{(\tilde{r} - \sigma^{\top} \xi - r)(T - t)}}
\end{aligned}

\begin{aligned}
\frac{1}{C} = e^{(\tilde{r} - \sigma^{\top} \xi - r)(T - t)}
\end{aligned}


Now, changing to some arbitrary measure 1 by inserting the Radon-Nikodym derivative into the expectation
$$
\begin{aligned}
& e^{-r(T - t)} X(0)\, \mathbb{E}_1\left[\,\left. \tilde{S}(T) \mathbf{I}_{\{X(T) > X(0)\}} \frac{\tilde{S}(T) \beta(T)}{\tilde{S}(T) \beta(t)} e^{(\tilde{r} - \sigma^{\top} \xi - r)(T - t)} \,\right|\, \mathcal{F}_t \right] \\
=\, & e^{(\tilde{r} - \sigma^{\top} \xi - r)(T - t)} X(0) \tilde{S}(t)\, \mathbb{Q}_1\left(X(T) > X(0) \right)
\end{aligned}
$$

We need the distribution of $X(T)$ under $Q_1$. Using Girsanov's thereom \
\begin{align*}
dW_1(t) &= dW_{\beta}(t) - \xi\, dt
\end{align*}

We can insert into the equation for X(T) $$
\begin{aligned}
& X(T)=X(t) e^{\left(r-\tilde{r}-\frac{1}{2} \|\sigma\|^2 +\sigma^{\top} \xi \right)(T-t) + \sigma\left(W_1(T)-W_1(t)\right)} \\
& \therefore \ln X(T) \left\lvert\, F_t \stackrel{Q_1}{\sim}\left(\ln X(t)+\left(r-\tilde{r}-\frac{1}{2} \|\sigma\|^2 +\sigma^{\top} \xi \right)(T-t), |ins|\sigma\|^2(T-t)\right)\right.
\end{aligned}
$$

Feeding that into the above probability, we have:
$$
e^{(\tilde{r}-\sigma^{\top} \xi -r)(T-t)} X(0) \tilde{S}(t) N \left( \frac{ \ln \left( \frac{X(t)}{X(0)} \right) + \left( r - \tilde{r} - \frac{1}{2} \|\sigma\|^2 + \sigma^{\top} \xi \right)(T - t) }{ \sigma \sqrt{T - t} } \right)
$$

So the price of the option is:
$$
= X(t) \tilde{S}(t) N \left( \frac{ \ln \left( \frac{X(t)}{X(0)} \right) + \left( r - \tilde{r} + \frac{1}{2} \|\sigma\|^2 + \sigma^{\top} \xi \right)(T - t) }{ \sigma \sqrt{T - t} } \right) \\
- e^{(\tilde{r}-\sigma^{\top} \xi -r)(T-t)} X(0) \tilde{S}(t) N \left( \frac{ \ln \left( \frac{X(t)}{X(0)} \right) + \left( r - \tilde{r} - \frac{1}{2} \|\sigma\|^2 + \sigma^{\top} \xi \right)(T - t) }{ \sigma \sqrt{T - t} } \right) \\
$$

Where $\sigma^{\top} \xi = \rho \sigma \xi$



In [7]:
import math
import numpy as np
from scipy.stats import norm
def Task3Option(S,X,r,r_f,T,volS,volX,rho):
    sqrt_T = math.sqrt(T)
    log_term = math.log(X / X)
    d1_num = log_term + (r - r_f + 0.5 * volX**2 + rho * volX * volS) * T
    d1 = d1_num / (volX * sqrt_T)
    d2_num = log_term + (r - r_f - 0.5 * volX**2 + rho * volX * volS) * T
    d2 = d2_num / (volX * sqrt_T)
    term1 = X * S * norm.cdf(d1)
    exp_term = math.exp((r_f - rho * volX * volS - r) * T)
    term2 = X * S * exp_term * norm.cdf(d2)
    return term1 - term2

In [8]:
S = float(input("Enter S~(0): "))
X = float(input("Enter X(0): "))
r = float(input("Enter r: "))
r_f = float(input("Enter r~: "))
T = float(input("Enter T: "))
volS = float(input("Enter volS: "))
volX = float(input("Enter volX: "))
rho = float(input("Enter rho: "))
price = Task3Option(S, X, r, r_f, T, volS, volX, rho)
print(f"The price of the option is {price}")

Enter S~(0): 100
Enter X(0): 1.5
Enter r: 0.04
Enter r~: 0.05
Enter T: 2
Enter volS: 0.3
Enter volX: 0.08
Enter rho: .25
The price of the option is 6.208394043356222


In [32]:
#taking the tutorial values as an example
Task3Option(100,1.5,0.04,0.05,2,0.3,0.08,0.25)

np.float64(6.208394043356222)

### Task 3(b)
Calculate the price of this option by Monte Carlo simulation and use this to verify that your implementation of Task 3(a) is correct. *(12 marks)*

In [33]:
def Task3OptionMC(S, X, r, r_f, T, volS, volX, rho, n):
    Z1 = np.random.standard_normal(n)
    Z2 = np.random.standard_normal(n) # start with two independent samples
    Z_S = Z1
    Z_X = rho * Z1 + np.sqrt(1 - rho**2) * Z2 # introduce the correlation of ZX with ZS.
    mu_S = r_f - rho * volS * volX - 0.5 * volS**2 # this is from the measure change from Beta~ to Beta, reflecting the quanto drift adjustment due to correlation.
    ST = S * np.exp(mu_S * T + volS * np.sqrt(T) * Z_S) #simulating ST. also note that that W~(0,T) so that's why we need to multiply Z_S by the square root time
    mu_X = r - r_f - 0.5 * volX**2 # under original Beta measure
    XT = X * np.exp(mu_X * T + volX * np.sqrt(T) * Z_X) # Simulating XT, already under the domestic risk neutral measure (Beta)
    payoff = np.maximum(0, XT*ST - X*ST)
    MC = np.exp(-r * T) * np.mean(payoff)
    MCstd = np.exp(-r * T) * np.std(payoff) / np.sqrt(n)
    return MC, MCstd,

In [35]:
foreignstock = float(input("Enter S~(0): "))
exchangerate = float(input("Enter X(0): "))
r = float(input("Enter r: "))
r_f = float(input("Enter r~: "))
Time = float(input("Enter T: "))
volS = float(input("Enter volS: "))
volX = float(input("Enter volX: "))
rho = float(input("Enter rho: "))
n = int(input('Enter the number of simulations: '))
MC, MCstd = Task3OptionMC(foreignstock, exchangerate, r, r_f, Time, volS, volX, rho, n)
print('The MC estimate for the option price is: ')
print(MC)
print('The MC standard deviation is: ')
print(MCstd)
print('The 2 standard deviation confidence interval for the option price is: ')
print(MC-2*MCstd,MC+2*MCstd)

Enter S~(0): 100
Enter X(0): 1.5
Enter r: 0.04
Enter r~: 0.05
Enter T: 2
Enter volS: 0.3
Enter volX: 0.08
Enter rho: 0.25
Enter the number of simulations: 100000
The MC estimate for the option price is: 
6.197271739207403
The MC standard deviation is: 
0.04063585021587798
The 2 standard deviation confidence interval for the option price is: 
6.116000038775646 6.278543439639159


In [36]:
np.random.seed(123) # fixing result by setting a seed

S = 100
X = 1.5
r = 0.04
r_f = 0.05
T = 2
volS = 0.3
volX = 0.08
rho = 0.25
n = 100000

mc_price, mc_error = Task3OptionMC(S, X, r, r_f, T, volS, volX, rho, n)
print(f"Monte Carlo price: {mc_price:.4f} ± {1.96 * mc_error:.4f} (95% CI)")
print(f"MC standard deviation: {mc_error:.4f}")

Monte Carlo price: 6.2342 ± 0.0797 (95% CI)
MC standard deviation: 0.0406


In [31]:
# Tutorial inputs for verification
S = 100
X = 1.5
r = 0.04
r_f = 0.05
T = 2
volS = 0.3
volX = 0.08
rho = 0.25

# Analytical price (Task 3(a))
analytical_price = Task3Option(S, X, r, r_f, T, volS, volX, rho)
print(f"Analytical price (Task 3(a)): {analytical_price:.6f}")

# MC price
n = 100000
mc_price, mc_std = Task3OptionMC(S, X, r, r_f, T, volS, volX, rho, n)
print(f"MC price (Task 3(b)): {mc_price:.6f} (Std Error: {mc_std:.6f})")

# Verification check
difference = abs(analytical_price - mc_price)
within_2_std = difference < 2 * mc_std
print(f"Difference: {difference:.6f}")
print(f"Within 2 Std Errors? {within_2_std}")

Analytical price (Task 3(a)): 6.208394
MC price (Task 3(b)): 6.203797 (Std Error: 0.040776)
Difference: 0.004597
Within 2 Std Errors? True


Since the difference between the Monte Carlo estimate and the analytical price is less than 2 standard errors, we can conclude that the Monte Carlo result is statistically consistent with/verifies the analytical solution from Task 3(a). This is because, under the assumptions of the central limit thereom and normally distributed noise, about 95% of Monte Carlo estimates are expected to fall within ±2 standard errors of the true value. We can therefore say that with 95% confidence, the interval given by the Monte Carlo estimate ±2 standard errors contains the true value.